# Capstone Two - Data Wrangling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn import preprocessing

### Data Collection

I'm following a population of students who matriculated into an institution in the 2015/2016 academic year, and graduated with a 4-year degree in the 2018/2019 academic year. 

In [2]:
matriculation_data = pd.read_csv('MERGED2015_16_PP.csv')

/var/folders/j4/96_5xmgd7j362v_w7_09bvb40000gn/T/ipykernel_1538/3904940649.py:1: DtypeWarning: Columns (1537,1540,1542,1561,1589,1590,1591,1601,1602,1606,1614,1615,1729,1909,1910,1911,1912,1913) have mixed types. Specify dtype option on import or set low_memory=False.
  matriculation_data = pd.read_csv('MERGED2015_16_PP.csv')


In [3]:
graduation_data = pd.read_csv('MERGED2018_19_PP.csv')

/var/folders/j4/96_5xmgd7j362v_w7_09bvb40000gn/T/ipykernel_1538/3099082543.py:1: DtypeWarning: Columns (1729,1909,1910,1911,1912,1913,2376,2377,2958) have mixed types. Specify dtype option on import or set low_memory=False.
  graduation_data = pd.read_csv('MERGED2018_19_PP.csv')


In [4]:
print("matriculation_data shape "+str(matriculation_data.shape))
print("graduation_data shape "+str(graduation_data.shape))

matriculation_data shape (7666, 2989)
graduation_data shape (6807, 2989)


In [5]:
a = matriculation_data['UNITID'].nunique()
b = graduation_data['UNITID'].nunique()
print(a, b)

7666 6807


The above block shows that the UNITID value is assigned uniquely to each institution.  We can inner join both data sets on UNITID to find the institutions relevant to our project.

In [6]:
data_merged = matriculation_data.merge(graduation_data, on="UNITID", how="inner", suffixes=("_met","_grad"))

I added _met_ and _grad_ suffixes to denote which columns came from which academic years in the merged df.

In [7]:
data_merged.shape

(6357, 5977)

Our merged data set preserved 6,357 unique institution observations.  

In [8]:
data_merged.head()

,UNITID,OPEID_met,OPEID6_met,INSTNM_met,CITY_met,STABBR_met,ZIP_met,ACCREDAGENCY_met,INSTURL_met,NPCURL_met,...,COUNT_WNE_MALE1_P8_grad,MD_EARN_WNE_MALE1_P8_grad,GT_THRESHOLD_P10_grad,MD_EARN_WNE_INC1_P10_grad,MD_EARN_WNE_INC2_P10_grad,MD_EARN_WNE_INC3_P10_grad,MD_EARN_WNE_INDEP1_P10_grad,MD_EARN_WNE_INDEP0_P10_grad,MD_EARN_WNE_MALE0_P10_grad,MD_EARN_WNE_MALE1_P10_grad
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,1233.0,49652.0,0.7472,42254.0,49817.0,51571.0,48182.0,46435.0,42007.0,56164.0
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,78.0,50355.0,0.6286,36636.0,44836.0,NaN,39040.0,NaN,32311.0,49599.0
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,891.0,57542.0,0.7769,49469.0,60533.0,57411.0,56884.0,53803.0,45170.0,66070.0
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,1077.0,32797.0,0.5178,30634.0,34533.0,38216.0,30602.0,32364.0,29836.0,35315.0


I am now importing population data by state from wikipedia to scale my data set.

In [9]:
states_url = 'https://simple.wikipedia.org/w/index.php?title=List_of_U.S._states&oldid=7168473'
usa_states = pd.read_html(states_url)
usa_states = usa_states[0]
usa_states.head()

Name & postal abbs. [1]                                 Cities               \
  Name & postal abbs. [1] Name & postal abbs. [1].1      Capital   Largest[5]   
0                 Alabama                        AL   Montgomery   Birmingham   
1                  Alaska                        AK       Juneau    Anchorage   
2                 Arizona                        AZ      Phoenix      Phoenix   
3                Arkansas                        AR  Little Rock  Little Rock   
4              California                        CA   Sacramento  Los Angeles   

  Established[A] Population [B][3] Total area[4]          Land area[4]  \
  Established[A] Population [B][3]           mi2      km2          mi2   
0   Dec 14, 1819           4903185         52420   135767        50645   
1    Jan 3, 1959            731545        665384  1723337       570641   
2   Feb 14, 1912           7278717        113990   295234       113594   
3   Jun 15, 1836           3017804         53179   137732        52035   
4    Sep 9, 1850          39512223        163695   423967       155779   

           Water area[4]         Number of Reps.  
       km2           mi2     km2 Number of Reps.  
0   131171          1775    4597               7  
1  1477953         94743  245384               1  
2   294207           396    1026               9  
3   134771          1143    2961               4  
4   403466          7916   20501              53

The features I'm interested in are from columns 1, 5, 6.

In [10]:
usa_states_sub = usa_states.iloc[:, [0,1,5,6]].copy()
usa_states_sub.columns = ["name", "abb","pop","area"]
usa_states_sub.head()

,name,abb,pop,area
0,Alabama,AL,4903185,52420
1,Alaska,AK,731545,665384
2,Arizona,AZ,7278717,113990
3,Arkansas,AR,3017804,53179
4,California,CA,39512223,163695


In [11]:
usa_states_sub.dtypes

name    object
abb     object
pop      int64
area     int64
dtype: object

In [12]:
usa_states_sub['name'] = usa_states_sub['name'].astype("string")
usa_states_sub['abb'] = usa_states_sub['abb'].astype("string")
usa_states_sub.dtypes

name    string
abb     string
pop      int64
area     int64
dtype: object

In [13]:
usa_states_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    50 non-null     string
 1   abb     50 non-null     string
 2   pop     50 non-null     int64 
 3   area    50 non-null     int64 
dtypes: int64(2), string(2)
memory usage: 1.7 KB


Checking to see if all states are accounted for in my merged dataset.

In [14]:
missing_states = set(data_merged['STABBR_met']) - set(usa_states_sub['abb'])
missing_states

{'AS', 'DC', 'FM', 'GU', 'MH', 'MP', 'PR', 'PW', 'VI'}

In [15]:
territories_url = 'https://simple.wikipedia.org/wiki/Territories_of_the_United_States'
usa_territories = pd.read_html(territories_url)
usa_territories = usa_territories[1]
usa_territories.head()

,Name,Abbr.,Location,Area,Population (2018),Capital,Largest town,Status,Acquired
0,American Samoa,AS,Polynesia (South Pacific),197.1 km2 (76 sq mi),50826,Pago Pago,Tafuna,"Unincorporated, American Samoa is de facto org...","April 17, 1900"
1,Guam,GU,Micronesia (North Pacific),543 km2 (210 sq mi),167772,Hagåtña,Dededo,"Unincorporated, organized","April 11, 1899"
2,Northern Mariana Islands,MP,Micronesia (North Pacific),463.63 km2 (179 sq mi),51994,Saipan,Garapan,"Unincorporated, organized (commonwealth)","November 4, 1986 (U.S.)"
3,Puerto Rico,PR,Caribbean (North Atlantic),"9,104 km2 (3,515 sq mi)",3294626,San Juan,San Juan,"Unincorporated, organized (commonwealth)","April 11, 1899"
4,Virgin Islands (U.S.),VI,Caribbean (North Atlantic),346.36 km2 (134 sq mi),106977,Charlotte Amalie,Charlotte Amalie,"Unincorporated, organized","March 31, 1917"


In [16]:
usa_territories_sub = usa_territories.iloc[:, [0,1,3,4]].copy()
usa_territories_sub.columns = ["name", "abb","area","pop"]
usa_territories_sub.head()

,name,abb,area,pop
0,American Samoa,AS,197.1 km2 (76 sq mi),50826
1,Guam,GU,543 km2 (210 sq mi),167772
2,Northern Mariana Islands,MP,463.63 km2 (179 sq mi),51994
3,Puerto Rico,PR,"9,104 km2 (3,515 sq mi)",3294626
4,Virgin Islands (U.S.),VI,346.36 km2 (134 sq mi),106977


In [17]:
usa_territories_sub[['name','abb','pop','area']]

,name,abb,pop,area
0,American Samoa,AS,50826,197.1 km2 (76 sq mi)
1,Guam,GU,167772,543 km2 (210 sq mi)
2,Northern Mariana Islands,MP,51994,463.63 km2 (179 sq mi)
3,Puerto Rico,PR,3294626,"9,104 km2 (3,515 sq mi)"
4,Virgin Islands (U.S.),VI,106977,346.36 km2 (134 sq mi)


In [18]:
usa_territories_sub['area']

0       197.1 km2 (76 sq mi)
1        543 km2 (210 sq mi)
2     463.63 km2 (179 sq mi)
3    9,104 km2 (3,515 sq mi)
4     346.36 km2 (134 sq mi)
Name: area, dtype: object

To save me a headache later, I will address the area objects by using .at to rename them according to their square mile distance and change the data type to integer.

In [19]:
usa_territories_sub.at[0, 'area'] = 76
usa_territories_sub.at[1, 'area'] = 210
usa_territories_sub.at[2, 'area'] = 179
usa_territories_sub.at[3, 'area'] = 3515
usa_territories_sub.at[4, 'area'] = 134
usa_territories_sub['area'].astype(int)

0      76
1     210
2     179
3    3515
4     134
Name: area, dtype: int64

In [20]:
missing_states = missing_states - set(usa_territories_sub['abb'])
missing_states

{'DC', 'FM', 'MH', 'PW'}

In [21]:
print(str(data_merged[data_merged['STABBR_met'] == 'DC']['STABBR_met'].count()) +" institutions in Washington D.C.")
print(str(data_merged[data_merged['STABBR_met'] == 'FM']['STABBR_met'].count()) +" institution in the Federated States of Micronesia ")
print(str(data_merged[data_merged['STABBR_met'] == 'MH']['STABBR_met'].count()) +" institution in the Marshall Islands")
print(str(data_merged[data_merged['STABBR_met'] == 'PW']['STABBR_met'].count()) +" institution in Republic of Palau")

23 institutions in Washington D.C.
1 institution in the Federated States of Micronesia 
1 institution in the Marshall Islands
1 institution in Republic of Palau


Other than Washington D.C., the remaining 3 institutions belong to territories that are related to USA by their US-supported military bases.  I plan to drop these three from the dataframe since they do not have a significant population overall and may likely provide outlier data to larger established institutions within continental USA. 

In [22]:
data_merged = data_merged[~data_merged['STABBR_met'].isin(['FM'])]
data_merged = data_merged[~data_merged['STABBR_met'].isin(['MH'])]
data_merged = data_merged[~data_merged['STABBR_met'].isin(['PW'])]

In [23]:
print(data_merged[data_merged['STABBR_met']== 'FM']['STABBR_met'].any())
print(data_merged[data_merged['STABBR_met']== 'MH']['STABBR_met'].any())
print(data_merged[data_merged['STABBR_met']== 'PW']['STABBR_met'].any())

False
False
False


The above block confirms that the three institutions are now removed from the dataset.  Now I'm going to add a row to territories to usa_territories_sub to include Washington D.C. as listed on https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States. 

In [24]:
df = {'name': 'Washington D.C.', 'abb': 'DC', 'area': 68, 'pop' : 689545}
usa_territories_sub = usa_territories_sub.append(df, ignore_index = True)
usa_territories_sub.head(6)

/var/folders/j4/96_5xmgd7j362v_w7_09bvb40000gn/T/ipykernel_1538/1198534965.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  usa_territories_sub = usa_territories_sub.append(df, ignore_index = True)


,name,abb,area,pop
0,American Samoa,AS,76,50826
1,Guam,GU,210,167772
2,Northern Mariana Islands,MP,179,51994
3,Puerto Rico,PR,3515,3294626
4,Virgin Islands (U.S.),VI,134,106977
5,Washington D.C.,DC,68,689545


In [25]:
usa_territories_sub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    6 non-null      object
 1   abb     6 non-null      object
 2   area    6 non-null      object
 3   pop     6 non-null      int64 
dtypes: int64(1), object(3)
memory usage: 320.0+ bytes


Now I'm going to concatonate usa_territories_sub and usa_states_sub.

In [26]:
pop_data = pd.concat([usa_territories_sub, usa_states_sub])

In [27]:
pop_data.reset_index()

,index,name,abb,area,pop
0,0,American Samoa,AS,76,50826
1,1,Guam,GU,210,167772
2,2,Northern Mariana Islands,MP,179,51994
3,3,Puerto Rico,PR,3515,3294626
4,4,Virgin Islands (U.S.),VI,134,106977
5,5,Washington D.C.,DC,68,689545
6,0,Alabama,AL,52420,4903185
7,1,Alaska,AK,665384,731545
8,2,Arizona,AZ,113990,7278717
9,3,Arkansas,AR,53179,3017804


I'm ready to merge with data_merged on abbreviations.

In [28]:
data_merged_2 = data_merged.merge(pop_data,left_on='STABBR_met', right_on="abb", how="left")
data_merged_2.rename(columns={"area": "state_or_territory_sq_mi", "pop": "state_or_territory_pop"})

,UNITID,OPEID_met,OPEID6_met,INSTNM_met,CITY_met,STABBR_met,ZIP_met,ACCREDAGENCY_met,INSTURL_met,NPCURL_met,...,MD_EARN_WNE_INC2_P10_grad,MD_EARN_WNE_INC3_P10_grad,MD_EARN_WNE_INDEP1_P10_grad,MD_EARN_WNE_INDEP0_P10_grad,MD_EARN_WNE_MALE0_P10_grad,MD_EARN_WNE_MALE1_P10_grad,name,abb,state_or_territory_sq_mi,state_or_territory_pop
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0,Alabama,AL,52420,4903185
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,49817.0,51571.0,48182.0,46435.0,42007.0,56164.0,Alabama,AL,52420,4903185
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,44836.0,NaN,39040.0,NaN,32311.0,49599.0,Alabama,AL,52420,4903185
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,60533.0,57411.0,56884.0,53803.0,45170.0,66070.0,Alabama,AL,52420,4903185
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,34533.0,38216.0,30602.0,32364.0,29836.0,35315.0,Alabama,AL,52420,4903185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6349,47691101,4205801,42058,SAE Institute of Technology San Francisco,Emeryville,CA,94608,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,California,CA,163695,39512223
6350,48065701,869423,8694,Rasmussen University - Overland Park,Overland Park,KS,66210-2786,NaN,NaN,NaN,...,40384.0,46825.0,34894.0,37892.0,33158.0,45403.0,Kansas,KS,82278,2913314
6351,48154401,4220901,42209,National Personal Training Institute of Cleveland,Highland Heights,OH,44143,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Ohio,OH,44826,11689100
6352,48387801,4223701,42237,Bay Area Medical Academy - San Jose Satellite ...,San Jose,CA,95113,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,California,CA,163695,39512223


### Data Definition

Now I want to investigate how many series have missing data.

In [29]:
print(data_merged_2.isnull().sum(axis = 0))

UNITID                           0
OPEID_met                        0
OPEID6_met                       0
INSTNM_met                       0
CITY_met                         0
                              ... 
MD_EARN_WNE_MALE1_P10_grad    2106
name                             0
abb                              0
area                             0
pop                              0
Length: 5981, dtype: int64


In [30]:
data_merged_2.dtypes

UNITID                          int64
OPEID_met                       int64
OPEID6_met                      int64
INSTNM_met                     object
CITY_met                       object
                               ...   
MD_EARN_WNE_MALE1_P10_grad    float64
name                           object
abb                            object
area                           object
pop                             int64
Length: 5981, dtype: object

In [31]:
data_merged_2['ADM_RATE_met'].describe()

count    1981.000000
mean        0.672838
std         0.206796
min         0.000000
25%         0.545500
50%         0.696000
75%         0.817600
max         1.000000
Name: ADM_RATE_met, dtype: float64

Here's the institution's admitions rate description of my student population from the 2015/2016 academic year.

In [32]:
data_merged_2['ACTCM25_met'].describe()

count    1231.000000
mean       20.440292
std         3.617635
min        12.000000
25%        18.000000
50%        20.000000
75%        22.000000
max        34.000000
Name: ACTCM25_met, dtype: float64

Here's the 25th percentile of the ACT cumulative scores description of my student population from the 2015/2016 academic year.

In [33]:
data_merged['C100_4_grad'].describe()

count    2036.000000
mean        0.365528
std         0.237919
min         0.000000
25%         0.183875
50%         0.341900
75%         0.531825
max         1.000000
Name: C100_4_grad, dtype: float64

Here's the student graduation rate for first-time, full-time students at four-year institutions of my student population from 2018/2019 academic year.

In [34]:
a = data_merged_2['ADM_RATE_met'].isnull().sum(axis = 0)
b = data_merged_2['ACTCM25_met'].isnull().sum(axis = 0)
c = data_merged_2['C100_4_grad'].isnull().sum(axis = 0)

print(str(a)+" institutions are missing their admissions rate data.")
print("")
print(str(b)+" institutions are missing their 25th percentile of the cumulative ACT score data.")
print("")
print(str(c)+" institutions are missing their first-time, full-time at four-year institutions graduation data.")

4373 institutions are missing their admissions rate data.

5123 institutions are missing their 25th percentile of the cumulative ACT score data.

4318 institutions are missing their first-time, full-time at four-year institutions graduation data.


### Data Cleaning

In [35]:
data_merged_2.rename(columns={'UNITID' :'Institution_ID',
    'INSTNM_met': 'institution_name',
                            'CITY_met': 'city',
                              'STABBR_met': 'state_or_territory_abbreviation',
                            'ZIP_met': 'zipcode',
                            'PREDDEG_grad':'predominant_degree_awarded',
                            'ADM_RATE_met':'admission_rate',
                            'ACTCM25_met':'act_25th_percentile_score',
                            'C100_4_grad':'graduation_rate',
                            'D100_4_grad': 'graduation_frequency',
                             'area':'territory_or_state_area_sqmi',
                             'pop':'territory_or_state_population'}, inplace=True)

In [36]:
data_merged_2.head()

,Institution_ID,OPEID_met,OPEID6_met,institution_name,city,state_or_territory_abbreviation,zipcode,ACCREDAGENCY_met,INSTURL_met,NPCURL_met,...,MD_EARN_WNE_INC2_P10_grad,MD_EARN_WNE_INC3_P10_grad,MD_EARN_WNE_INDEP1_P10_grad,MD_EARN_WNE_INDEP0_P10_grad,MD_EARN_WNE_MALE0_P10_grad,MD_EARN_WNE_MALE1_P10_grad,name,abb,territory_or_state_area_sqmi,territory_or_state_population
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0,Alabama,AL,52420,4903185
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,49817.0,51571.0,48182.0,46435.0,42007.0,56164.0,Alabama,AL,52420,4903185
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,44836.0,NaN,39040.0,NaN,32311.0,49599.0,Alabama,AL,52420,4903185
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,60533.0,57411.0,56884.0,53803.0,45170.0,66070.0,Alabama,AL,52420,4903185
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,34533.0,38216.0,30602.0,32364.0,29836.0,35315.0,Alabama,AL,52420,4903185


In [37]:
data_merged_reduced = data_merged_2[['Institution_ID',
                                   'institution_name',
                                     'city',
                                     'state_or_territory_abbreviation',
                                    'territory_or_state_area_sqmi',
                                     'territory_or_state_population',
                          'predominant_degree_awarded',
                                   'admission_rate',
                                   'act_25th_percentile_score',
                                   'graduation_rate',
                                   'graduation_frequency',
                                    ]]

In [38]:
data_merged_reduced.head()

,Institution_ID,institution_name,city,state_or_territory_abbreviation,territory_or_state_area_sqmi,territory_or_state_population,predominant_degree_awarded,admission_rate,act_25th_percentile_score,graduation_rate,graduation_frequency
0,100654,Alabama A & M University,Normal,AL,52420,4903185,3,0.6538,16.0,0.0556,756.0
1,100663,University of Alabama at Birmingham,Birmingham,AL,52420,4903185,3,0.6043,22.0,0.3469,1652.0
2,100690,Amridge University,Montgomery,AL,52420,4903185,3,NaN,NaN,0.4000,10.0
3,100706,University of Alabama in Huntsville,Huntsville,AL,52420,4903185,3,0.8120,24.0,0.2195,615.0
4,100724,Alabama State University,Montgomery,AL,52420,4903185,3,0.4639,16.0,0.0975,1436.0


In [39]:
data_merged_reduced.shape

(6354, 11)

### I need to drop all records of institutions who do no grant at least a BA degree.

In [40]:
data_merged_reduced = data_merged_reduced[data_merged_reduced['predominant_degree_awarded']>=3]

### Continuing to remove records with missing important data.

In [41]:
data_merged_reduced = data_merged_reduced[data_merged_reduced['act_25th_percentile_score'].notna()]
data_merged_reduced.isnull().sum()

Institution_ID                     0
institution_name                   0
city                               0
state_or_territory_abbreviation    0
territory_or_state_area_sqmi       0
territory_or_state_population      0
predominant_degree_awarded         0
admission_rate                     1
act_25th_percentile_score          0
graduation_rate                    8
graduation_frequency               8
dtype: int64

In [42]:
data_merged_reduced = data_merged_reduced[data_merged_reduced['graduation_rate'].notna()]
data_merged_reduced.isnull().sum()

Institution_ID                     0
institution_name                   0
city                               0
state_or_territory_abbreviation    0
territory_or_state_area_sqmi       0
territory_or_state_population      0
predominant_degree_awarded         0
admission_rate                     0
act_25th_percentile_score          0
graduation_rate                    0
graduation_frequency               0
dtype: int64

In [43]:
data_merged_reduced.shape

(1187, 11)

After removing institutions with critical missing data, I have 1187 observations remaining.  I need to check datatypes and normalize data next.

In [44]:
data_merged_reduced.dtypes

Institution_ID                       int64
institution_name                    object
city                                object
state_or_territory_abbreviation     object
territory_or_state_area_sqmi        object
territory_or_state_population        int64
predominant_degree_awarded           int64
admission_rate                     float64
act_25th_percentile_score          float64
graduation_rate                    float64
graduation_frequency               float64
dtype: object

In [45]:
data_merged_reduced['institution_name'] = data_merged_reduced['institution_name'].astype('string')
data_merged_reduced['city'] = data_merged_reduced['city'].astype('string')
data_merged_reduced['state_or_territory_abbreviation'] = data_merged_reduced['state_or_territory_abbreviation'].astype('string')
data_merged_reduced['territory_or_state_area_sqmi'] = data_merged_reduced['territory_or_state_area_sqmi'].astype('int')
data_merged_reduced['Institution_ID'] = data_merged_reduced['Institution_ID'].astype('object')
data_merged_reduced['predominant_degree_awarded'] = data_merged_reduced['predominant_degree_awarded'].astype('string')

In [46]:
data_merged_reduced.dtypes

Institution_ID                      object
institution_name                    string
city                                string
state_or_territory_abbreviation     string
territory_or_state_area_sqmi         int64
territory_or_state_population        int64
predominant_degree_awarded          string
admission_rate                     float64
act_25th_percentile_score          float64
graduation_rate                    float64
graduation_frequency               float64
dtype: object

Now I will normalize the data set.

In [47]:
data_merged_reduced= data_merged_reduced[['Institution_ID', 'institution_name', 'city', 'state_or_territory_abbreviation', 'predominant_degree_awarded', 'territory_or_state_area_sqmi', 'territory_or_state_population', 'admission_rate', 'act_25th_percentile_score', 'graduation_rate', 'graduation_frequency']]
data_merged_reduced['predominant_degree_awarded'] = data_merged_reduced.predominant_degree_awarded.str.replace('3', 'Bachelor\'s Degree')
data_merged_reduced['predominant_degree_awarded']= data_merged_reduced.predominant_degree_awarded.str.replace('4', 'Graduate Degree')
data_merged_reduced.head()

,Institution_ID,institution_name,city,state_or_territory_abbreviation,predominant_degree_awarded,territory_or_state_area_sqmi,territory_or_state_population,admission_rate,act_25th_percentile_score,graduation_rate,graduation_frequency
0,100654,Alabama A & M University,Normal,AL,Bachelor's Degree,52420,4903185,0.6538,16.0,0.0556,756.0
1,100663,University of Alabama at Birmingham,Birmingham,AL,Bachelor's Degree,52420,4903185,0.6043,22.0,0.3469,1652.0
3,100706,University of Alabama in Huntsville,Huntsville,AL,Bachelor's Degree,52420,4903185,0.8120,24.0,0.2195,615.0
4,100724,Alabama State University,Montgomery,AL,Bachelor's Degree,52420,4903185,0.4639,16.0,0.0975,1436.0
5,100751,The University of Alabama,Tuscaloosa,AL,Bachelor's Degree,52420,4903185,0.5359,22.0,0.5049,6302.0


In [48]:
data_merged_reduced.to_csv('data_cleaned_non_norm.csv')

In [49]:
data_merged_reduced.iloc[:, 5:] = data_merged_reduced.iloc[:, 5:].apply(lambda x: (x-x.mean())/x.std(), axis=0)
data_merged_reduced.head()

,Institution_ID,institution_name,city,state_or_territory_abbreviation,predominant_degree_awarded,territory_or_state_area_sqmi,territory_or_state_population,admission_rate,act_25th_percentile_score,graduation_rate,graduation_frequency
0,100654,Alabama A & M University,Normal,AL,Bachelor's Degree,-0.329042,-0.654946,-0.007511,-1.249341,-1.796206,-0.207178
1,100663,University of Alabama at Birmingham,Birmingham,AL,Bachelor's Degree,-0.329042,-0.654946,-0.260956,0.408841,-0.354295,0.494820
3,100706,University of Alabama in Huntsville,Huntsville,AL,Bachelor's Degree,-0.329042,-0.654946,0.802489,0.961569,-0.984914,-0.317649
4,100724,Alabama State University,Montgomery,AL,Bachelor's Degree,-0.329042,-0.654946,-0.979817,-1.249341,-1.588804,0.325588
5,100751,The University of Alabama,Tuscaloosa,AL,Bachelor's Degree,-0.329042,-0.654946,-0.611170,0.408841,0.427793,4.138001


In [50]:
data_merged_reduced.columns

Index(['Institution_ID', 'institution_name', 'city',
       'state_or_territory_abbreviation', 'predominant_degree_awarded',
       'territory_or_state_area_sqmi', 'territory_or_state_population',
       'admission_rate', 'act_25th_percentile_score', 'graduation_rate',
       'graduation_frequency'],
      dtype='object')

In [51]:
data_merged_reduced.to_csv('data_cleaned.csv')